In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [1]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=9f61cfd51268b88f05fbc4b5e1ca01ed6287127ae12cce195e7d20090912507f
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [3]:
import plotly.graph_objs as go
import datetime as dt
import pandas as pd
#import talib as ta
import pandas_ta as ta

name = '積水ハウス'
code = 1928
df = get_stock_data(code)
close = df['Close']

#ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
ma5, ma25 = ta.sma(close ,length=5), ta.sma(close ,length=25)
df['ma5'], df['ma25'] = ma5, ma25

rdf = df[dt.datetime(2021,12,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime('%m-%d-%Y')

layout = {
            "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
            "xaxis" : { "title": "日付", "rangeslider": { "visible": False },  "type" : "category" },
            "yaxis" : { "title": "価格（円）", "side": "left", "tickformat": "," },
            "plot_bgcolor":"light blue"
          }

data =  [
            # ローソク足
            go.Candlestick(x=rdf.index, open=rdf["Open"], high=rdf["High"], low=rdf["Low"], close=rdf["Close"],   
                            increasing_line_color="red", decreasing_line_color="gray"),
            # 5日移動平均線
            go.Scatter(x=rdf.index, y=rdf["ma5"], name="MA5", line={ "color": "royalblue", "width": 1.2 } ),
            # 25日移動平均線
            go.Scatter(x=rdf.index, y=rdf["ma25"], name="MA25", line={ "color": "lightseagreen", "width": 1.2 } )
        ]

fig = go.Figure(data = data, layout = go.Layout(layout))
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()